#Importing necessary libraries

In [37]:
import pandas as pd
import numpy as np
import csv
from collections import Counter
import copy

#Loading the dataset

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')
FILE_DIR='/content/gdrive/MyDrive/Fall 2024/Data Warehousing'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Reading from CSV file

In [39]:
data = pd.read_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/Final_DF.csv")
df = data.copy(deep=True)
df.drop(df.columns[[0]], axis=1, inplace=True)
df

,ID,Name,Address,Name of Colleges,Major,Degree,GPA,Semester,Y,Status,Month Day,Name of Old_Colleges
0,330,GG. Art,Savannah,College of Art and Science,English,BA,3.1,Summer 4,91,N,Aug 15,College of Science
1,522,BB. Mercy,Savannah,College of Art and Science,English,MA,4.0,Summer 4,91,N,Aug 15,College of Science
2,403,C. Brown,Denver,Cyber College,Applied Sc,PhD,4.0,Fall,89,I,Dec. 15,Computer Communications
3,667,K. Chen,Conway,College of Education,Elementary Ed,EdD,3.2,Summer 4,92,O,Aug 15,College of Education
4,529,M. Yu,Conway,College of Education,Secondary Ed,MS,3.2,Spring,90,I,May. 15,College of Education
...,...,...,...,...,...,...,...,...,...,...,...,...
126,785,Y. Bankrpt,Conway,Cyber College,Applied Sc,MS,3.2,Summer 1,87,O,June 1,Computer Communications
127,413,UI. Wagon,Little Rock,Cyber College,Computer Sc,MS,3.1,Summer 1,84,O,June 1,Computer Communications
128,898,C. Browner,Denver,Cyber College,Applied Sc,MS,4.0,Spring,89,O,May. 15,Computer Communications
129,777,F. Anglic,Atlanta,College of Art and Science,Biology,BS,3.9,Spring,90,N,May. 15,College of Science


#Name of Columns

In [40]:
columns_list = df.columns.tolist()
columns_list

['ID',
 'Name',
 'Address',
 'Name of Colleges',
 'Major',
 'Degree',
 'GPA',
 'Semester',
 'Y',
 'Status',
 'Month Day',
 'Name of Old_Colleges']

#Creating CSV files for the Dimensions

GPA_Status dimension

In [41]:
gpa_status_df=df[['GPA']].copy()
#gpa_status_df
gstatus_id=[]
gpa_standing=[]
gpa=gpa_status_df['GPA'].tolist()

for i in range(len(gpa)):
  if float(gpa[i])> 4.0:
    gpa[i]=4.0
  if 4.0>= float(gpa[i])  and float(gpa[i]>=3.0):
    gpa_standing.append("High")
    gstatus_id.append("G1")
  elif 3.0 > float(gpa[i]) and float(gpa[i]) > 2.0:
    gpa_standing.append("Medium")
    gstatus_id.append("G2")
  elif 2.0>= float(gpa[i]):
    gpa_standing.append("Low")
    gstatus_id.append("G3")
  else:
    print(gpa[i])
    gpa_standing.append("NULL")
    gstatus_id.append("NULL")

gpa_status_df.insert(0, 'GPAstatus_ID', gstatus_id)
gpa_status_df.insert(2, 'GPA_Standing', gpa_standing)
#print(len(gpa))
#print(len(gpa_standing))
#print(len(gstatus_id))
#print(gpa_status_df[gpa_status_df['GPA']<=2.0])
ugpa_status_df = gpa_status_df.drop_duplicates().reset_index(drop=True)
ugpa_status_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UGPA_statusDF.csv",index=False)
ugpa_status_df


,GPAstatus_ID,GPA,GPA_Standing
0,G1,3.1,High
1,G1,4.0,High
2,G1,3.2,High
3,G1,3.9,High
4,G2,2.2,Medium
5,G1,3.8,High
6,G2,2.9,Medium
7,G1,4.1,High
8,G2,2.1,Medium
9,G1,3.3,High


Student dimension

In [42]:
student_df = df[['ID','Name','Address']].copy()
student_df.insert(3, 'GPAstatus_ID', gstatus_id)
student_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UStudentDF.csv",index=False)
student_df

,ID,Name,Address,GPAstatus_ID
0,330,GG. Art,Savannah,G1
1,522,BB. Mercy,Savannah,G1
2,403,C. Brown,Denver,G1
3,667,K. Chen,Conway,G1
4,529,M. Yu,Conway,G1
...,...,...,...,...
126,785,Y. Bankrpt,Conway,G1
127,413,UI. Wagon,Little Rock,G1
128,898,C. Browner,Denver,G1
129,777,F. Anglic,Atlanta,G1


Status Dimension

In [43]:
status_df=df[['Status']].copy()
#gpa_status_df
status_id=[]
status_desc=[]
status=status_df['Status'].tolist()

for i in range(len(status)):
  if status[i]== "I":
    status_id.append("S1")
    status_desc.append("International Student")
  elif status[i]== "O":
    status_id.append("S2")
    status_desc.append("Out of state")
  elif status[i]== "N":
    status_id.append("S3")
    status_desc.append("Non-of-the above")
  else:
    print(status)
    status_id.append("NULL")
    status_desc.append("NULL")

status_df.insert(0, 'status_id', status_id)
status_df.insert(2, 'status_desc', status_id)
ustatus_df = status_df.drop_duplicates().reset_index(drop=True)
ustatus_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UStatusDF.csv", index=False)
ustatus_df


,status_id,Status,status_desc
0,S3,N,S3
1,S1,I,S1
2,S2,O,S2


Year Dimension

In [44]:
year_df=df[['Y']].copy()
year_id=[]
year=year_df['Y'].tolist()
year_count=Counter(year)
year_keys= list(year_count.keys())
year_keys.sort()
#print(year_keys)
for i in range(len(year)):
  for j in range(len(year_keys)):
    if year[i]==year_keys[j]:
      year_id.append(f"Y{j+1}")

#print(len(year))
#print(len(year_id))
year_df.insert(0, 'year_id', year_id)
uyear_df = year_df.drop_duplicates().reset_index(drop=True)
uyear_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UYearDF.csv", index=False)
uyear_df

,year_id,Y
0,Y8,91
1,Y6,89
2,Y9,92
3,Y7,90
4,Y4,87
5,Y1,84
6,Y5,88
7,Y3,86
8,Y2,85


Semester Dimension

In [45]:
sem_df=df[['Semester', 'Month Day',]].copy()
sem_id=[]
semester=sem_df['Semester'].tolist()
sem_value = sorted(list(sem_df['Semester'].unique()))
sem_value
for i in range(len(semester)):
  for j in range(len(sem_value)):
    if semester[i]==sem_value[j]:
      sem_id.append(f"sem{j+1}")

print(len(semester))
print(len(sem_id))
sem_df.insert(0, 'sem_id', sem_id)
sem_df.insert(3,'year_id', year_id)
usem_df = sem_df.drop_duplicates().reset_index(drop=True)
usem_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/USemesterDF.csv",index=False)
usem_df

131
131


,sem_id,Semester,Month Day,year_id
0,sem6,Summer 4,Aug 15,Y8
1,sem1,Fall,Dec. 15,Y6
2,sem6,Summer 4,Aug 15,Y9
3,sem2,Spring,May. 15,Y7
4,sem4,Summer 2,July 4,Y4
5,sem4,Summer 2,July 4,Y1
6,sem3,Summer 1,June 1,Y4
7,sem1,Fall,Dec. 15,Y8
8,sem2,Spring,May. 15,Y1
9,sem2,Spring,May. 15,Y4


College Dimension

In [46]:
#['ID', 'Name', 'Address', 'Name of Colleges', 'Major', 'Degree', 'GPA', 'Semester','Y', 'Status', 'Month Day','Name of Old_Colleges']
college_df=df[['Name of Colleges', 'Name of Old_Colleges',]].copy()
c_id=[]
c_name=college_df['Name of Colleges'].tolist()
name_value = list(college_df['Name of Colleges'].unique())
print(name_value)
for i in range(len(c_name)):
  for j in range(len(name_value)):
    if c_name[i]==name_value[j]:
      c_id.append(f"C{j+1}")

print(len(c_name))
print(len(c_id))
college_df.insert(0, 'C_id', c_id)
ucollege_df = college_df.drop_duplicates().reset_index(drop=True)
ucollege_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UCollegeDF.csv",index=False)
ucollege_df


['College of Art and Science', 'Cyber College', 'College of Education', 'College of Business']
131
131


,C_id,Name of Colleges,Name of Old_Colleges
0,C1,College of Art and Science,College of Science
1,C2,Cyber College,Computer Communications
2,C3,College of Education,College of Education
3,C4,College of Business,College of Business and IS


Major Dimension

In [47]:
#['ID', 'Name', 'Address', 'Name of Colleges', 'Major', 'Degree', 'GPA', 'Semester','Y', 'Status', 'Month Day','Name of Old_Colleges']
major_df=df[['Major']].copy()
major_id=[]
major_name=major_df['Major'].tolist()
major_value = list(major_df['Major'].unique())
print(major_value)
for i in range(len(major_name)):
  for j in range(len(major_value)):
    if major_name[i]==major_value[j]:
      major_id.append(f"M{j+1}")

print(len(major_name))
print(len(major_id))
major_df.insert(0, 'Major_id', major_id)
major_df.insert(2, 'College_id', c_id)
umajor_df = major_df.drop_duplicates().reset_index(drop=True)
umajor_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UMajorDF.csv", index=False)
umajor_df

['English', 'Applied Sc', 'Elementary Ed', 'Secondary Ed', 'Computer Sc', 'Biology', 'Economics', 'Business Admin', 'Accounting', 'Information Sc', 'Chemistry']
131
131


,Major_id,Major,College_id
0,M1,English,C1
1,M2,Applied Sc,C2
2,M3,Elementary Ed,C3
3,M4,Secondary Ed,C3
4,M5,Computer Sc,C2
5,M6,Biology,C1
6,M7,Economics,C4
7,M8,Business Admin,C4
8,M9,Accounting,C4
9,M10,Information Sc,C2


Degree Dimension

In [48]:
#['ID', 'Name', 'Address', 'Name of Colleges', 'Major', 'Degree', 'GPA', 'Semester','Y', 'Status', 'Month Day','Name of Old_Colleges']
degree_df=df[['Degree']].copy()
degree_id=[]
degree_name=degree_df['Degree'].tolist()
degree_value = list(degree_df['Degree'].unique())
print(degree_value)
for i in range(len(degree_name)):
  for j in range(len(degree_value)):
    if degree_name[i]==degree_value[j]:
      degree_id.append(f"D{j+1}")

print(len(degree_name))
print(len(degree_id))
degree_df.insert(0, 'Degree_id', degree_id)
degree_df.insert(2, 'Major_id', major_id)
udegree_df = degree_df.drop_duplicates().reset_index(drop=True)
udegree_df.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/UDegreeDF.csv", index=False)
udegree_df

['BA', 'MA', 'PhD', 'EdD', 'MS', 'BS', 'AS', 'MBA']
131
131


,Degree_id,Degree,Major_id
0,D1,BA,M1
1,D2,MA,M1
2,D3,PhD,M2
3,D4,EdD,M3
4,D5,MS,M4
5,D4,EdD,M4
6,D6,BS,M5
7,D7,AS,M6
8,D5,MS,M5
9,D6,BS,M6


Fact Table

In [52]:
fstd_id = df['ID'].tolist()
fsem_id=sem_df['sem_id'].tolist()
fstatus_id=status_df['status_id'].tolist()
fdegree_id=degree_df['Degree_id'].tolist()
list= []
for i in range(len(fstd_id)):
  lst=[]
  lst.append(fstd_id[i])
  lst.append(fsem_id[i])
  lst.append(fstatus_id[i])
  lst.append(fdegree_id[i])
  list.append(lst)

fact_graduate=pd.DataFrame(list, columns=['std_id','sem_id','status_id','degree_id'])
fact_graduate.to_csv("/content/gdrive/MyDrive/Fall 2024/Data Warehousing/Assignment/Part3/CSV/Fact_graduate.csv", index=False)

fact_graduate




,std_id,sem_id,status_id,degree_id
0,330,sem6,S3,D1
1,522,sem6,S3,D2
2,403,sem1,S1,D3
3,667,sem6,S2,D4
4,529,sem2,S1,D5
...,...,...,...,...
126,785,sem3,S2,D5
127,413,sem3,S2,D5
128,898,sem2,S2,D5
129,777,sem2,S3,D6
